# Constrained optimization

Julia has its own modeling language, `JuMP.jl`, kind of like GAMS or AMPL. If you write your problem using this interface, you can use GOBS of different solvers to solve your problem. Even better, Julia automatically cooks up derivatives using automagic differentiation (reverse mode, I think...). It also figures out what the sparsity patterns in your Hessian are, which is super nice, too. Unfortunately, the API isn't super-well documented and can be kind of tedious to use. The code below should help you if you want to use `JuMP` on the midterm or to code up BLP or whatever.

JuMP: <https://jump.dev/JuMP.jl/stable/>

JuMP guide on nonlinear programming (which is most of what we do): <https://jump.dev/JuMP.jl/stable/nlp/>

MOI back end for JuMP: <https://jump.dev/MathOptInterface.jl/stable/>

Ipopt: is an open-source constrained optimization software <https://coin-or.github.io/Ipopt/>. If performanced is an issue, you can look in to having Ipopt use a different solver for the sparse linear equations.

# Constrained optimiazion approach for Nonrenewable Rscs

Here's a simple Hotelling model that you've seen earlier in lecture, and that you'll see again in ~~275~~ ARE 277

$$
\max_{\{q_t\}_{t=0}^T} \sum_{t=0}^T \beta^t \log q_t \qquad st \qquad Q \geq \sum_t q_t
$$

In [ ]:
# JuMP is Julia's linear + nonlinear programming language
using JuMP

# Ipopt is the "free" version of Knitro
using Ipopt

# for working w/ Hessian
using LinearAlgebra, SparseArrays

# Kind of cute if you want to make a nice table
using DataFrames

In [ ]:
n = 10
β = 0.9
Q = 5.0

model = Model(with_optimizer(Ipopt.Optimizer))  # define empty model solved by Ipopt algorithm
@variable(model, q[i=1:n] >= 0)
@constraint(model, sum(q[i] for i in 1:n) <= Q)
@NLobjective(model, Max, sum(log(q[i])*β^(i-1) for i in 1:n))

@show optimize!(model)

In [ ]:
# can call JuMP.value with the variable name to output it
value.(q)

Modify the model so that we have

$$
\max_{\{q_t,R_{t+1}\}_{t=0}^T} \sum_{t=0}^T \beta^t \left\{q_t - c(q_t,R_t)\right\} \qquad st \qquad R_t - q_t \geq R_{t+1}
$$


In [ ]:
n = 8
β = 0.9
Q = 10.0

model = Model(with_optimizer(Ipopt.Optimizer))  # define empty model solved by Ipopt algorithm

# define variables chosen by solver
@variable(model, q[t=1:n] >= 0)
@variable(model, R[t=1:n+1] >= 0)

# Starting stock is constrained
@constraint(model, R[1] == Q)

# Stock transition
@constraint(model, R[1:n] .- q[1:n] .>= R[2:n+1])

# you can write "expressions" and re-use them
@NLexpression(model, u[t=1:n], q[t] - 10*q[t]/R[t])

# the objective function
@NLobjective(model, Max, sum( u[t]*β^(t-1) for t in 1:n))

# solve it
@show optimize!(model)

In [ ]:
DataFrame(q = vcat(value.(q),0), R=value.(R))

In [ ]:
# get all the variables / values
var_names = all_variables(model)
var_hats = value.(var_names)
DataFrame(nms = var_names, vals = var_hats)

In [ ]:
# if we want to get gradient or Hessian, 
# need to use the MathOptInterface to get under the hood
d = JuMP.NLPEvaluator(model)
MOI.initialize(d, [:Grad, :Hess])

In [ ]:
@show nvar = num_variables(model)

grad = zeros(nvar)
MOI.eval_objective_gradient(d, grad, var_hats)
grad

Documentation for the functions below is at

<https://jump.dev/MathOptInterface.jl/stable/apireference/#NLP-evaluator-methods-1>

In [ ]:
# structure of hessian is sparse
hess_sparsity = MOI.hessian_lagrangian_structure(d)

In [ ]:
# initalize
H = zeros(length(hess_sparsity))

# evaluate D²(f(θ) + ∑ᵢ λᵢ gᵢ(θ))
λ = zeros(0)
MOI.eval_hessian_lagrangian(d, H, var_hats, 1.0, λ)

"makes dense hessian from sparse lower-triangular hessian"
function dense_hessian(hessian_sparsity, V, n)
    I = [i for (i,j) in hessian_sparsity]
    J = [j for (i,j) in hessian_sparsity]
    raw = sparse(I, J, V, n, n)
    return Matrix(raw + raw' - sparse(diagm(0=>diag(raw))))
end

# dense Hessian
hess = dense_hessian(hess_sparsity, H, nvar)